Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [140]:
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed, choice
from scipy import sparse
from copy import copy
import numpy as np

In [141]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

### Make Sets

In [142]:
# Define some constants
MAX_ITERATIONS = 10_000
NUM_POINTS = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [0.3, 0.7]
INDEX = 0
CALLS = 0

In [143]:
# fitnes function
def fitness(state, sets):
    global CALLS
    CALLS += 1
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [sets[:, [i]].toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_SETS[INDEX])]),
        )
    )
    return valid, -cost

In [144]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS[INDEX] - 1)
    new_state[index] = not new_state[index]
    return new_state

In [145]:
sets = make_set_covering_problem(NUM_POINTS[INDEX], NUM_SETS[INDEX], DENSITY[INDEX])
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS[INDEX])]

for step in range(MAX_ITERATIONS):
    new_state = tweak(current_state)
    fit1 = fitness(new_state, sets)
    fit2 = fitness(current_state, sets)
    if fit1 >= fit2:
        current_state = new_state
        print(f"Step {step}: {fit2}")
        
print(f'Number of calls to fitness function: {CALLS}')

Step 0: (0, 0)
Step 1: (2600, -1)
Step 2: (4800, -2)
Step 3: (6400, -3)
Step 4: (7200, -4)
Step 5: (8200, -5)
Step 6: (8900, -6)
Step 7: (9200, -7)
Step 8: (9500, -8)
Step 13: (9700, -9)
Step 15: (9700, -8)
Step 19: (9900, -9)
Step 23: (10000, -10)
Step 82: (10000, -9)
Number of calls to fitness function: 20000
